In [1]:
#import numpy as np 
import pandas as pd 
import numpy as np
import tensorflow as tf
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from statistics import mean

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
#parameters 
n_steps = 5
units = 20 
layers = 4
drop = 0.2
epochs = 1
batch_size = 10
optimizer = 'adam'
second_variable = 'residential_percent_change_from_baseline'
second_varible_metric = 'mean'
dim_test = 5

In [ ]:
"""PART1 - DATA PROCESSING"""

#import dataset

df = pd.read_csv('/home/luis/workspace/covid_visualization/dataset.csv')
print(df)
X = df  #Guarda a partir da coluna 3 (4) até a penultima
X_1 = X.groupby(['week_predicton'])['last_week_cases'].agg('sum')
X_1 = pd.DataFrame(X_1)
sc_1 = MinMaxScaler(feature_range = (0,1)) 
X_1 = sc_1.fit_transform(X_1)

X_2 = X.groupby(['week_predicton'])[second_variable].agg(second_varible_metric)
X_2 = pd.DataFrame(X_2)
sc_2 = MinMaxScaler(feature_range = (0,1)) 
X_2 = sc_2.fit_transform(X_2)

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


# define input sequence
in_seq1 = X_1
in_seq2 = X_2
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# choose a number of time steps
n_steps = n_steps
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
# choose a number of time steps
n_steps = n_steps
# convert into input/output
X, y = split_sequences(dataset, n_steps)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
X_test = X[-dim_test:]
y_test = y[-dim_test:]
X = X[:-dim_test]
y = y[:-dim_test]


In [ ]:
"""PART2 - BUILDING THE LSTM"""

# Adding the first LSTM layer and some dropout regularization
model = Sequential()
model.add(LSTM(units = units, return_sequences = True, input_shape=(n_steps, n_features)))
model.add(Dropout(drop))

# Adding the second LSTM layer and some dropout regularization
i = 0
while i < layers:
    model.add(LSTM(units = units, return_sequences = True))
    model.add(Dropout(drop))
    i += 1

model.add(LSTM(units = units, return_sequences = False))
model.add(Dropout(drop))

#Adding the output layer 
model.add(Dense(units = 1))





In [ ]:
"""Part3 - Training The LSTM """

# Compiling the RNN 
model.compile(optimizer = optimizer, loss = 'mean_squared_error')

#Fitting the RNN to Training set

history = model.fit(X, y, epochs = epochs, batch_size = batch_size)


In [ ]:
print(min(history.history['loss']))

In [ ]:
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc_1.inverse_transform(predicted_stock_price)
y_test = sc_1.inverse_transform(array(y_test.reshape(1, -1)))
y_test = y_test[0]
# Vizualizing the Results
plt.plot(y_test, color = 'red', label = 'Real')
plt.plot(predicted_stock_price, color = 'green', label = 'Predicted')
plt.title('Covid Predicton')
plt.xlabel('week')
plt.ylabel('cases')
plt.legend()
plt.show()

print(abs(sum((predicted_stock_price-y_test)**2)/dim_test))